In [ ]:
import os
import datetime
import firebirdsql
from queue import Queue
import concurrent.futures
from openpyxl import Workbook
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

In [ ]:
# ===================== Funções de Conexão =====================
def get_firebird_connection():
    # Ajuste os parâmetros conforme sua configuração, inclusive charset
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=int(os.getenv('PORT', '3050')),
        database=os.getenv('DB_PATH'),
        user=os.getenv('APP_USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )

In [ ]:
def create_connection_pool(pool_size=20):
    """
    Cria um pool de conexões Firebird com pool_size fixo (20).
    """
    pool = Queue(maxsize=pool_size)
    for _ in range(pool_size):
        conn = get_firebird_connection()
        pool.put(conn)
    return pool

def get_connection_from_pool(pool):
    # Fica bloqueado até ter uma conexão disponível no pool
    return pool.get()

def release_connection_to_pool(pool, conn):
    # Devolve a conexão ao pool
    pool.put(conn)
    
# ===================== Função para obter a quantidade total de registros =====================
def get_total_count(pool):
    """
    Retorna o número total de registros da tabela PRODUTO.
    """
    conn = get_connection_from_pool(pool)
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM PRODUTO")
        count = cursor.fetchone()[0]
        return count
    finally:
        release_connection_to_pool(pool, conn)

In [ ]:
# ===================== Função para carregar similares =====================
def load_similars(pool):
    """
    Executa a query para capturar os similares e monta um dicionário:
      chave: p.NUMORIGINAL (produto)
      valor: lista de s.NUMORIGINAL (similares)
    """
    conn = get_connection_from_pool(pool)
    try:
        cursor = conn.cursor()
        query = """
            SELECT
                p.NUMORIGINAL,  
                s.NUMORIGINAL
            FROM SIMILARIDADE s
            JOIN PRODUTO p ON p.CDPRODUTO = s.CDPRODUTO
            ORDER BY p.NUMORIGINAL
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        similar_dict = {}
        for prod_num, similar_num in rows:
            # Converte para string, caso não esteja
            prod_key = str(prod_num)
            similar_val = str(similar_num)
            if prod_key in similar_dict:
                similar_dict[prod_key].append(similar_val)
            else:
                similar_dict[prod_key] = [similar_val]
        return similar_dict
    finally:
        release_connection_to_pool(pool, conn)

In [ ]:
# ===================== Função para buscar um "chunk" de registros =====================
def fetch_chunk2(offset, chunk_size, pool):
    """
    Busca um pedaço (chunk) de registros da tabela PRODUTO.
    """
    conn = get_connection_from_pool(pool)
    try:
        cursor = conn.cursor()
        query = (
            "SELECT FIRST {} SKIP {} NUMORIGINAL, ESTOQUEPREVISTO, PRECOCUSTO, PRECOVENDA, DESCRICAO, LOCALIZACAO FROM PRODUTO"
        ).format(chunk_size, offset)
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows
    finally:
        release_connection_to_pool(pool, conn)

In [ ]:
# ===================== Função para buscar um "chunk" de registros =====================
def fetch_chunk(offset, chunk_size, pool):
    """
    Busca um pedaço (chunk) de registros da tabela PRODUTO.
    """
    conn = get_connection_from_pool(pool)
    try:
        cursor = conn.cursor()
        query = (
            "SELECT FIRST {} SKIP {} CDPRODUTO, NUMORIGINAL, DESCRICAO, PRECOCUSTO, PRECOVENDA, "
            "ESTOQUEPREVISTO, UNIDADE, NCM, LOCALIZACAO FROM PRODUTO "
            "ORDER BY CDPRODUTO"
        ).format(chunk_size, offset)
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows
    finally:
        release_connection_to_pool(pool, conn)

In [ ]:
# ===================== Mapeamento de cada registro para a linha do Excel =====================
def process_record2(db_row, current_date_str, similar_dict):
    """
    Recebe uma tupla (db_row) com os campos extraídos da tabela PRODUTO:
      db_row = (NUMORIGINAL, ESTOQUEPREVISTO, PRECOCUSTO, PRECOVENDA, DESCRICAO, LOCALIZACAO)

    Retorna uma lista com os valores mapeados conforme as regras:
      - Campo 0: NUMORIGINAL
      - Campo 1: ESTOQUEPREVISTO
      - Campo 2: PRECOCUSTO
      - Campo 3: PRECOVENDA
      - Campo 4: DESCRICAO
      - Campo 5: LOCALIZACAO
    """
    
    # Converte o NUMORIGINAL para string
    numoriginal = str(db_row[0])
    
    # Verifica se o produto tem similares
    similares = similar_dict.get(numoriginal, [])
    
    # Mapeia os campos
    return [
        numoriginal,
        db_row[1],
        db_row[2],
        db_row[3],
        db_row[4],
        db_row[5]
    ]

In [ ]:
# ===================== Mapeamento de cada registro para a linha do Excel =====================
def process_record(db_row, current_date_str, similar_dict):
    """
    Recebe uma tupla (db_row) com os campos extraídos da tabela PRODUTO:
      db_row = (CDPRODUTO, NUMORIGINAL, DESCRICAO, PRECOCUSTO, PRECOVENDA,
                ESTOQUEPREVISTO, UNIDADE, NCM, LOCALIZACAO)

    Retorna uma lista com os valores mapeados conforme as regras:
      - Campo 0 (CODIGO SEQUENCIAL): vazio
      - Campo 1 (Nome da Empresa): "comagro"
      - Campo 2 (CODIGO SEQUENCIAL DO ITEM): CDPRODUTO
      - Campo 3 (NOME GRUPO): "TEMPORARIO"
      - Campo 4 (NOME DESCRICAO): "TEMPORARIO"
      - Campo 5 (NOME FABRICANTE): "DIVERSOS"
      - Campo 6 (Nº FABRICANTE): NUMORIGINAL
      - Campo 7 (CÓDIGO DE BARRAS - EAN 13): "SEM GTIN"
      - Campo 8 (CÓDIGO DE BARRAS - qualquer formato): "SEM GTIN"
      - Campo 9 (APLICAÇÃO DO PRODUTO): DESCRICAO
      - Campo 10 (INFORMAÇÕES ADICIONAIS): string de similares no formato "SIMILARES:xxx,yyy,zzz"
      - Campo 11 (Alíquota de IPI): 0
      - Campo 12 (Situação tributária do IPI): "060"
      - Campo 13 (Alíquota de ICMS na Entrada): vazio
      - Campo 14 (Peso da peça): vazio
      - Campo 15 (Unidade da peça): UNIDADE
      - Campo 16 (Código ANP): vazio
      - Campo 17 (Quantidade Embalagem de Compra): 1
      - Campo 18 (Quantidade Embalagem de Venda): 1
      - Campo 19 (Classificação Fiscal): NCM
      - Campo 20 (Preço de tabela): vazio
      - Campo 21 (Data do Preço de tabela): vazio
      - Campo 22 (Data do Cadastro): data atual
      - Campo 23 (Origem do Produto): 0
      - Campo 24 (Situação Tributária do Item): "00"
      - Campo 25 (Percentual da Alíquota ICMS): "20,5%"
      - Campo 26 (Preço Custo): PRECOCUSTO
      - Campo 27 (Preço venda): PRECOVENDA
      - Campo 28 (Quantidade Estoque): ESTOQUEPREVISTO
      - Campo 29 (Posição fixa do item): LOCALIZACAO
      - Campos 30 a 37: vazios
    """
    # Captura o produto (NUMORIGINAL) para buscar os similares
    prod_num = str(db_row[1])
    similares = similar_dict.get(prod_num, [])
    if similares:
        similar_field = "SIMILARES:" + ",".join(similares)
    else:
        similar_field = ""

    return [
        "",                   # 0. CODIGO SEQUENCIAL (vazio)
        "COMAGRO",            # 1. Nome da Empresa
        db_row[0],            # 2. CODIGO SEQUENCIAL DO ITEM (CDPRODUTO)
        "TEMPORARIO",         # 3. NOME GRUPO
        "TEMPORARIO",         # 4. NOME DESCRICAO
        "DIVERSOS",           # 5. NOME FABRICANTE
        db_row[1],            # 6. Nº FABRICANTE (NUMORIGINAL)
        "SEM GTIN",           # 7. CÓDIGO DE BARRAS (EAN 13)
        "SEM GTIN",           # 8. CÓDIGO DE BARRAS (qualquer formato)
        db_row[2],            # 9. APLICAÇÃO DO PRODUTO (DESCRICAO)
        similar_field,        # 10. INFORMAÇÕES ADICIONAIS (similares)
        0,                    # 11. Alíquota de IPI
        "060",                # 12. Situação tributária do IPI
        "",                   # 13. Alíquota de ICMS na Entrada (vazio)
        "",                   # 14. Peso da peça (vazio)
        db_row[6],            # 15. Unidade da peça (UNIDADE)
        "",                   # 16. Código ANP (vazio)
        1,                    # 17. Quantidade Embalagem de Compra
        1,                    # 18. Quantidade Embalagem de Venda
        db_row[7],            # 19. Classificação Fiscal (NCM)
        "",                   # 20. Preço de tabela (vazio)
        "",                   # 21. Data do Preço de tabela (vazio)
        current_date_str,     # 22. Data do Cadastro
        0,                    # 23. Origem do Produto
        "00",                 # 24. Situação Tributária do Item
        "20,5%",              # 25. Percentual da Alíquota ICMS
        db_row[3],            # 26. Preço Custo (PRECOCUSTO)
        db_row[4],            # 27. Preço venda (PRECOVENDA)
        db_row[5],            # 28. Quantidade Estoque (ESTOQUEPREVISTO)
        db_row[8],            # 29. Posição fixa do item (LOCALIZACAO)
        "",                   # 30. Margem de Lucro do item (vazio)
        "",                   # 31. Mkup para preço atacado (vazio)
        "",                   # 32. Mkup para preço varejo (vazio)
        "",                   # 33. Mkup Mínimo (vazio)
        "",                   # 34. Estoque crítico (vazio)
        "",                   # 35. Estoque máximo (vazio)
        "",                   # 36. Quantidade de dias (máximo) (vazio)
        ""                    # 37. Quantidade de dias (mínimo) (vazio)
    ]

In [ ]:

# ===================== Função Principal =====================
def main():
    # Cria pool de conexões
    pool = create_connection_pool(pool_size=20)
    
    # Carrega os similares (dicionário: chave = p.NUMORIGINAL, valor = lista de similares)
    similar_dict = load_similars(pool)
    
    # Obtém a quantidade total de registros da tabela PRODUTO
    total_count = get_total_count(pool)
    print(f"Total de registros a extrair: {total_count}")
    
    # Define o tamanho do chunk (pode ser ajustado conforme volume de dados)
    chunk_size = 100
    offsets = range(0, total_count, chunk_size)
    
    # Data atual formatada (usada para "Data do Cadastro")
    current_date_str = datetime.datetime.now().strftime("%d/%m/%Y")
    
    all_data_rows = []  # aqui serão armazenadas todas as linhas processadas
    
    # Usa ThreadPoolExecutor para processar os chunks em paralelo
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        # Mapeia cada tarefa (chunk) com seu offset
        future_to_offset = {
            executor.submit(fetch_chunk, offset, chunk_size, pool): offset
            for offset in offsets
        }
        for future in concurrent.futures.as_completed(future_to_offset):
            offset = future_to_offset[future]
            try:
                chunk = future.result()
                print(f"Processado chunk com offset {offset} (tamanho: {len(chunk)})")
                # Para cada registro do chunk, mapeia para a linha do Excel
                for db_row in chunk:
                    excel_row = process_record(db_row, current_date_str, similar_dict)
                    all_data_rows.append(excel_row)
            except Exception as exc:
                print(f"Erro ao processar chunk com offset {offset}: {exc}")
    
    print(f"Total de registros processados: {len(all_data_rows)}")
    
    # ===================== Criação do Excel =====================
    wb = Workbook()
    ws = wb.active
    
    # Deixa as duas primeiras linhas em branco (para que você insira os cabeçalhos manualmente)
    ws.append([])
    ws.append([])
    
    # Insere as linhas de dados (cada linha já com 38 colunas)
    for row in all_data_rows:
        ws.append(row)
    
    # Salva o arquivo Excel
    output_filename = "produtos-170225.xlsx"
    wb.save(output_filename)
    print(f"Arquivo Excel salvo como '{output_filename}'")
    
    # Fecha todas as conexões do pool
    while not pool.empty():
        conn = pool.get()
        conn.close()

if __name__ == "__main__":
    main()